### Extract embedding axis

In [ ]:
# !pip install --upgrade accelerate transformers

In [2]:
import pandas as pd

In [3]:
# Open training_embeddings.csv and load it into a dataframe
df = pd.read_csv('training_embeddings.csv')
df.head()

,word,0,1,2,3,4,5,6,7,8,...,758,759,760,761,762,763,764,765,766,767
0,companion,0.128,-0.082,0.109,-0.012,0.317,0.262,-0.016,0.128,0.357,...,0.395,-0.107,-0.026,-0.061,0.358,-0.138,-0.067,0.236,0.385,-0.115
1,toast,0.209,0.411,0.026,-0.000,-0.243,-0.325,0.148,0.238,-0.120,...,0.373,-0.174,0.119,-0.036,0.445,-0.029,0.145,0.169,0.364,-0.097
2,lounge,0.759,-0.116,0.116,0.113,0.460,-0.212,0.115,-0.032,0.137,...,0.259,0.070,-0.059,-0.114,0.441,-0.096,-0.074,0.196,0.215,-0.254
3,watch,0.401,-0.003,-0.061,-0.406,0.933,-0.140,-0.186,0.286,-0.170,...,0.459,-0.067,-0.266,-0.318,0.173,-0.109,0.219,-0.010,0.404,-0.161
4,haul,0.580,0.031,0.311,0.048,-0.102,0.081,0.047,0.423,-0.187,...,0.505,0.096,0.095,-0.009,0.453,0.003,0.337,0.259,0.064,0.092


In [19]:
def get_x_embedding(df, x):
  """
  Extract the x column from the dataframe and store it as a list
  """
  # Extract the word and x column from the dataframe and store them as a list of lists
  # index = 0
  words = df['word'].tolist()
  emd_index = df[str(x)].tolist()
  # data = [[words[i], emd_index[i]] for i in range(len(words))]
  # print(data[:5])

  # Concatenate the two lists into a list of lists
  data = [words,emd_index]
  return data

data = get_x_embedding(df, 0)
print(len(data), len(data[0]), len(data[1]))
print(data[0][:5])
print(data[1][:5])

2 383 383
['companion', 'toast', 'lounge', 'watch', 'haul']
[0.128, 0.209, 0.759, 0.401, 0.58]


### Get axis interpretation from OpenAI

In [ ]:
# install OpenAI api
# !pip install --upgrade openai

In [21]:
# load api key from secrets.json
import openai
import json

try:
    with open("secrets.json") as f:
        secrets = json.load(f)
    my_api_key = secrets["openai"]
    print("API key loaded.")
    openai.api_key = my_api_key
except FileNotFoundError:
    print("Secrets file not found. YOU NEED THEM TO RUN THIS.")

API key loaded.


In [24]:
axis_emb = get_x_embedding(df, 0)

In [32]:
# model_num = "gpt-3.5-turbo-1106"
# model_num = "gpt-4-1106-preview"
model_num = "gpt-4"

completion = openai.ChatCompletion.create(
  model=model_num,
  messages=[
    {"role": "system", "content": "You are an expert transformer embeddings labeller."},
    {"role": "user", "content": f"Below are two lists. The first list contains words that have been put into DistilBERT. DistilBERT creates an embedding with 768 dimensions or axes. The second list contains the embedding value from DistilBERT for one axis across the words. By carefully comparing and considering the embedding values for each word, please interpret the likely linguistic feature that this embedding axis encodes. This interpretation must be consistent across all the words and correspond to their respective positive, zero, or negative embedding values.\n\n  The output must be a Python dictionary with three items, the key is a string containing the possible interpretation of the axis. The value is a float, representing the confidence score from 0 to 1. \n\n Here is an output sample: {{<first interpretation>:<first interpretation confidence score>, <second interpretation>:<second interpretation confidence score>, <third interpretation>:<third interpretation confidence score>}} \n\n {axis_emb[0]}\n\n {axis_emb[1]}"}
  ]
)

print(completion.choices[0].message)

# log the stringified output into a txt file by appending it to the end of the file
with open("output.txt", "a") as f:
  f.write(str(completion))

{
  "role": "assistant",
  "content": "{\"Verb or Action Related\": 0.7, \"Instrument or Tools Associated\": 0.6, \"Emotion or Feeling Connotated\": 0.5}"
}


In [57]:
# convert the output string into a dictionary
interp_dict = eval(completion.choices[0].message.content)
print(interp_dict)

# convert the dictionary keys into a list
interp_keys = list(interp_dict.keys())
print(interp_keys[0])

{'Verb or Action Related': 0.7, 'Instrument or Tools Associated': 0.6, 'Emotion or Feeling Connotated': 0.5}
Verb or Action Related


### Evaluate interpretations using OpenAI

In [69]:
# # test the prompt using smaller set
# trunc_word_list = axis_emb[0][:50]
# print(trunc_word_list)

['companion', 'toast', 'lounge', 'watch', 'haul', 'combination', 'majesty', 'extinguish', 'nutrient', 'bark', 'rest', 'hold', 'drainage', 'chill', 'time', 'drink', 'eat', 'journey', 'surf', 'shipment', 'measure', 'devour', 'cutlery', 'catch', 'drive', 'cooling', 'sail', 'flutter', 'moo', 'descend', 'commute', 'score', 'indulgence', 'cutting', 'containment', 'communication', 'transportation', 'glide', 'slice', 'drying', 'tote', 'nurture', 'breakfast', 'cut', 'throw', 'brush', 'competition', 'signal', 'carry', 'mobility']


In [74]:
import time

In [89]:
# model_num = "gpt-3.5-turbo-1106"
# model_num = "gpt-4-1106-preview"
model_num = "gpt-4"

score_list = []
step = 30

for i in range((len(axis_emb[0])// step )+ 1):
  # test to make sure the list is being truncated correctly
  # print(i*step, (i+1)*step)
  trunc_word_list = axis_emb[0][i*step:(i+1)*step]
  # print(trunc_word_list)
  # print(len(trunc_word_list))

  is_length_correct = False

  while (is_length_correct == False):
    print("\n\nStep ", i + 1, " of ", (len(axis_emb[0])// step )+ 1)
    print(i+1, " Input length: ", len(trunc_word_list))

    completion = openai.ChatCompletion.create(
      model=model_num,
      messages=[
        {"role": "system", "content": "You are an expert word sense scorer."},
        {"role": "user", "content": f"For the list of words below, please assign it a score according to how much it relates to the following criteria: '{interp_keys[0]}' \n\n  The output must be a Python list of scores for each corresponding word in the provided list. The output must therefore have {len(trunc_word_list)} items, the length as the provided list. The score is a float that ranges from -1 to 1. Positive scores suggest a high correlation to the criteria, while negative scores suggest a high opposite correlation. Scores closer to 0 suggest that the criterion is not applicable to the word. \n\n Here is an output sample: [<score for first word>, <score for second word>, ... , <score for second-last word>, <score for last word>] \n\n {trunc_word_list}"}
        # {"role": "user", "content": f"For the list of words below, please assign it a score according to how much it relates to the following criteria: {interp_keys[0]}  \n\n  The output must be a Python list of scores for each corresponding word in the provided list. The output must therefore have the same number of items as the provided list. The score is a float that ranges from -1 to 1. Positive scores suggest a high correlation to the criteria, while negative scores suggest a high opposite correlation. Scores closer to 0 suggest that the criterion is not applicable to the word. \n\n Here is an output sample: [<score for first word>, <score for second word>, ... , <score for second-last word>, <score for last word>] \n\n {axis_emb[0]}"}
      ]
    )

    # print(completion.choices[0].message)

    # log the stringified output into a txt file by appending it to the end of the file
    with open("eval_output.txt", "a") as f:
      f.write(str(completion))

    # convert the output string into a list
    try:
      scores = eval(completion.choices[0].message.content)
    except:
      print(i+1, "Error: ", completion.choices[0].message.content)
      print("Trying again...")
      continue
    print(scores)

    # check if the length is correct
    print(i+1, "Output length: ", len(scores))
    print(i+1, "Are input output lengths the same? " ,len(scores) == len(trunc_word_list))

    if len(scores) == len(trunc_word_list):
      is_length_correct = True
    else:
      print("Input output lengths are not the same. Trying again...")
      continue

    # concatenate scores with score_list
    score_list += scores

    # giving it more time – does it lead to better results?
    time.sleep(10)

with open("eval_output.txt", "a") as f:
  f.write("\nscore_list: " + str(score_list))
print("Length of embedding list: ", len(axis_emb[0]))
print("Length of score list: ", len(score_list))



Step  1  of  13
1  Input length:  30
[0, 0.65, -0.1, 0.7, 0.8, -0.3, -0.5, 0.8, -0.6, 0.4, 0.6, 0.75, -0.15, 0.65, -0.35, 0.7, 0.9, 0.7, 0.8, -0.2, 0.85, 0.9, -0.6, 0.75, 0.95, -0.6, 0.9, 0.8, -0.45, 0.85]
1 Output length:  30
1 Are input output lengths the same?  True


Step  2  of  13
2  Input length:  30
[1.0, 1.0, -0.5, 1.0, -0.5, 0.2, 0.5, 1.0, 1.0, 1.0, 1.0, 0.8, 0.3, 1.0, 1.0, 1.0, -0.2, 0.5, 1.0, 0.2, 0.8, -0.8, -0.8, -0.5, 1.0, 1.0, 0.9, 0.3, 0, 1.0]
2 Output length:  30
2 Are input output lengths the same?  True


Step  3  of  13
3  Input length:  30
[-1, -1, 0.9, 0.8, -1, 0.2, -1, -1, -1, 1, -0.5, -0.2, -1, 1, -0.5, 0.9, 1, -0.9, 1, -0.9, -1, 1, 1, 1, -0.9, -0.3, 1, 0.8, 1, 1]
3 Output length:  30
3 Are input output lengths the same?  True


Step  4  of  13
4  Input length:  30
[1.0, 1.0, -0.1, 1.0, -0.2, -0.1, 1.0, -0.1, 1.0, 1.0, 1.0, -1.0, -0.7, 0.2, -1.0, 0.7, 1.0, -0.3, 1.0, 1.0, -0.1, 1.0, -0.8, -1.0, -0.8, 0.2, -0.4, -1.0, -1.0, -1.0]
4 Output length:  30
4 Are inpu

In [113]:
# compare score_list with axis_emb[1] by subtracting them
diff_list = [score_list[i] - axis_emb[1][i] for i in range(len(score_list))]
print(diff_list[:5])

# abs and round the difference list to 3 decimal places
diff_list = [abs(round(diff, 3)) for diff in diff_list]

# sum the difference list
sum_diff = sum(diff_list)
print("Sum of diff: ", sum_diff)

# calculate the mean of the difference list
mean_diff = sum(diff_list)/len(diff_list)
print("Mean of diff: ", mean_diff)

with open("eval_output.txt", "a") as f:
  f.write("\ndiff_list: " + str(diff_list) + "\nsum of diff: " + str(sum_diff) + "\nmean of diff: " + str(mean_diff) + "\n\n")

[-0.128, 0.44100000000000006, -0.859, 0.29899999999999993, 0.22000000000000008]
Sum of diff:  284.1839999999997
Mean of diff:  0.7419947780678843


Matte's notes:
- mean difference of 0.74 is unacceptable

### Comparison between two runs + evaluating the approach

In [114]:
# compare between two runs

first_run = [0.0, 0.7, -0.3, 0.6, 0.8, -0.2, -0.9, 0.9, -0.9, 0.7, 0.6, 0.8, -0.1, 0.7, 0.0, 0.7, 0.8, 0.8, 0.8, 0.5, 0.7, 1.0, -0.3, 0.9, 0.9, -0.3, 0.8, 0.7, 0.3, 1.0, 0.9, 0.8, -0.5, 0.8, -0.4, 0.2, 0.6, 0.9, 0.9, 0.6, 0.8, 0.4, 0.1, 1.0, 1.0, 0.9, 0.2, 0.6, 0.9, 0.1, 0.4, -0.8, -0.9, -0.4, 0.8, 0.9, 0.7, 0.3, 0.1, 0.6, 0.1, -0.2, 0.7, 0.7, -0.6, 0.2, -0.3, -0.4, -0.3, 1.0, -0.2, -0.1, -0.3, 1.0, 0.2, 0.7, 1.0, -0.5, 1.0, -0.6, -0.1, 1.0, 1.0, 1.0, 0.1, 0.2, 1.0, 0.6, 0.8, 1.0, 1.0, 1.0, -0.6, 1.0, -0.8, -0.5, 0.2, -0.4, 1.0, 1.0, 1.0, -0.9, -0.6, -0.4, -1.0, 0.2, 1.0, -0.8, 1.0, 0.6, -0.3, 1.0, -0.9, -1.0, -0.7, 0.6, -0.4, -1.0, -1.0, -1.0, -0.2, -0.4, -0.1, 0.5, -0.3, -0.4, 0.8, 0.7, 0.95, -0.5, 0.9, 0.85, -0.25, 0.95, 0.7, -0.2, 0.6, 0.95, 0.8, -0.8, -0.2, -0.1, -0.1, -0.15, -0.17, -0.12, -0.02, -0.02, -0.85, -0.95, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 0.2, 0.5, 0.5, 0.3, 0.3, 0.4, 0.4, 0.6, 0.6, 0.5, -0.5, -0.5, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 1.0, -1.0, -0.5, -1.0, -1.0, 0.8, -0.8, 1.0, 0.8, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, 0.0, 0.0, -0.2, -0.2, -1.0, -1.0, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 0, 0, 0.9, 0.9, 0.9, 0.9, 0.9, 0, 0, 0, 0.7, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.75, 0.75, 0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.7, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.8, 0.0, 0.0, 0.0, 0.6, 0.7, 0.7, 0.6, 0.7, -1, 0.3, 0.3, 0.3, -1, -1, -1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0.6, 0.6, 1, 1, 1, 1, 1, 1, 0.6, 0.6, 1, 1, 1, 1, 0.8, 0.7, 0.8, 0.9, 0.7, 0.9, -0.2, -0.2, -0.2, -0.2, -0.2, -0.1, -0.1, -0.2, -0.2, -0.3, -0.1, -0.1, 0.3, -0.1, -0.1, -1, -1]
second_run = [0, 0.65, -0.1, 0.7, 0.8, -0.3, -0.5, 0.8, -0.6, 0.4, 0.6, 0.75, -0.15, 0.65, -0.35, 0.7, 0.9, 0.7, 0.8, -0.2, 0.85, 0.9, -0.6, 0.75, 0.95, -0.6, 0.9, 0.8, -0.45, 0.85, 1.0, 1.0, -0.5, 1.0, -0.5, 0.2, 0.5, 1.0, 1.0, 1.0, 1.0, 0.8, 0.3, 1.0, 1.0, 1.0, -0.2, 0.5, 1.0, 0.2, 0.8, -0.8, -0.8, -0.5, 1.0, 1.0, 0.9, 0.3, 0, 1.0, -1, -1, 0.9, 0.8, -1, 0.2, -1, -1, -1, 1, -0.5, -0.2, -1, 1, -0.5, 0.9, 1, -0.9, 1, -0.9, -1, 1, 1, 1, -0.9, -0.3, 1, 0.8, 1, 1, 1.0, 1.0, -0.1, 1.0, -0.2, -0.1, 1.0, -0.1, 1.0, 1.0, 1.0, -1.0, -0.7, 0.2, -1.0, 0.7, 1.0, -0.3, 1.0, 1.0, -0.1, 1.0, -0.8, -1.0, -0.8, 0.2, -0.4, -1.0, -1.0, -1.0, 0, -0.5, 0, -0.3, -0.3, -0.7, 0.8, 0.9, 1, -0.3, 0.8, 0.9, -0.5, 1, 0.9, -0.4, -0.2, 1, 1, -1, -0.6, -0.7, -0.6, -0.8, -0.7, -0.7, -0.8, -0.6, -0.9, -0.9, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -0.2, -0.2, -0.2, -0.3, -0.2, 0.6, 0.7, 0.7, 0.5, 0.6, 0.8, 0.7, 0.7, 0.8, 0.8, -0.4, -0.4, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 1, -1, -0.2, -1, -1, 0.8, -0.5, 0.7, 0.5, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -0.4, -0.4, -0.4, -0.4, -1, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, -1, 0.8, 0.8, 0.8, 0.9, 0.9, -1, -1, -1, -1, 0.7, -1, -1, -1, -1, -1, -1, 0.5, -1, -1, -1, -1, 0.1, 0.2, -1, 0.3, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 0.5, -1, -1, -1, -1, -1, -1, -1, 0.7, -1, -1, -1, 1, 1, 1, 1, -1, -1, 0.5, 0.5, 0.5, -1, -1, -1, 1, 1, 1, 1, 0.5, 0.8, 0.8, 0.8, 1, 0.4, 0.4, 1, 1, 1, 0.8, 0.7, 1, 0.3, 0.3, 0.8, 1, 0.5, 1, 0.1, 0.1, 0.3, 0.6, 0.1, 0.6, -0.3, -0.3, -0.3, -0.3, -0.3, 0.2, 0.2, -0.3, -0.3, -0.3, 0.2, 0.2, 0.4, 0.2, 0.2, -0.5, -0.5]

print("Length of first run: ", len(first_run))
print("Length of second run: ", len(second_run)) 

diff_run_list = [first_run[i] - second_run[i] for i in range(len(first_run))]

# abs and round the difference list to 3 decimal places
diff_run_list = [abs(round(diff, 3)) for diff in diff_run_list]
print(diff_run_list)

sum_diff_run = sum(diff_run_list)
print("Sum of diff between two runs: ", sum_diff_run)
mean_diff_run = sum(diff_run_list)/len(diff_run_list)
print("Mean of diff between two runs: ", mean_diff_run)

Length of first run:  383
Length of second run:  383
[0.0, 0.05, 0.2, 0.1, 0.0, 0.1, 0.4, 0.1, 0.3, 0.3, 0.0, 0.05, 0.05, 0.05, 0.35, 0.0, 0.1, 0.1, 0.0, 0.7, 0.15, 0.1, 0.3, 0.15, 0.05, 0.3, 0.1, 0.1, 0.75, 0.15, 0.1, 0.2, 0.0, 0.2, 0.1, 0.0, 0.1, 0.1, 0.1, 0.4, 0.2, 0.4, 0.2, 0.0, 0.0, 0.1, 0.4, 0.1, 0.1, 0.1, 0.4, 0.0, 0.1, 0.1, 0.2, 0.1, 0.2, 0.0, 0.1, 0.4, 1.1, 0.8, 0.2, 0.1, 0.4, 0.0, 0.7, 0.6, 0.7, 0.0, 0.3, 0.1, 0.7, 0.0, 0.7, 0.2, 0.0, 0.4, 0.0, 0.3, 0.9, 0.0, 0.0, 0.0, 1.0, 0.5, 0.0, 0.2, 0.2, 0.0, 0.0, 0.0, 0.5, 0.0, 0.6, 0.4, 0.8, 0.3, 0.0, 0.0, 0.0, 0.1, 0.1, 0.6, 0.0, 0.5, 0.0, 0.5, 0.0, 0.4, 0.2, 0.0, 0.1, 0.0, 0.1, 0.4, 0.0, 0.0, 0.0, 0.0, 0.2, 0.1, 0.1, 0.8, 0.0, 0.3, 0.0, 0.2, 0.05, 0.2, 0.1, 0.05, 0.25, 0.05, 0.2, 0.2, 0.8, 0.05, 0.2, 0.2, 0.4, 0.6, 0.5, 0.65, 0.53, 0.58, 0.78, 0.58, 0.05, 0.05, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.8, 0.8, 0.8, 0.7, 0.8, 0.4, 0.2, 0.2, 0.2, 0.3, 0.4, 0.3, 0.1, 0.2, 0.3, 0.1, 0.1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

Matte's notes:
- 0.35 difference between 2 runs seems ok but it can probably be better

In [111]:
# create dataframe of words, embedding values, and scores
df = pd.DataFrame({'word': axis_emb[0], 'embedding': axis_emb[1], 'score': first_run})
print(df.head())

# save dataframe as csv
df.to_csv('llm-outputs/first_run.csv', index=False)

        word  embedding  score
0  companion      0.128    0.0
1      toast      0.209    0.7
2     lounge      0.759   -0.3
3      watch      0.401    0.6
4       haul      0.580    0.8


In [112]:
# create dataframe of words, embedding values, and scores
df = pd.DataFrame({'word': axis_emb[0], 'embedding': axis_emb[1], 'score': second_run})
print(df.head())

# save dataframe as csv
df.to_csv('llm-outputs/second_run.csv', index=False)

        word  embedding  score
0  companion      0.128   0.00
1      toast      0.209   0.65
2     lounge      0.759  -0.10
3      watch      0.401   0.70
4       haul      0.580   0.80


Matte's notes:
- it seems like the results are somewhat repeatable

In [118]:
# surface words from axis_emb[0] that have a difference of more than 0.5
len_a = 0
for i in range(len(diff_run_list)):
  if diff_run_list[i] > 0.75:
    print(axis_emb[0][i], "|(", first_run[i], ")", "-", "(", second_run[i], ")|", "=", diff_run_list[i])
    len_a += 1

print("Number of words with a difference of more than 0.5 (not confident): ", len_a, "of", len(diff_run_list))
print("Percentage: ", round(len_a/len(diff_run_list) * 100), "%")

health |( 0.1 ) - ( -1 )| = 1.1
entertainment |( -0.2 ) - ( -1 )| = 0.8
reflection |( -0.1 ) - ( -1 )| = 0.9
grace |( 0.1 ) - ( -0.9 )| = 1.0
heating |( 0.2 ) - ( 1.0 )| = 0.8
dry |( 0.5 ) - ( -0.3 )| = 0.8
prong |( 0.6 ) - ( -0.2 )| = 0.8
truck |( -0.02 ) - ( -0.8 )| = 0.78
backpack |( -1 ) - ( -0.2 )| = 0.8
umbrella |( -1 ) - ( -0.2 )| = 0.8
handbag |( -1 ) - ( -0.2 )| = 0.8
suitcase |( -1 ) - ( -0.2 )| = 0.8
one |( -1 ) - ( 0 )| = 1
two |( -1 ) - ( 0 )| = 1
10 |( -1 ) - ( 0 )| = 1
the |( -1 ) - ( 0 )| = 1
a |( -1 ) - ( 0 )| = 1
The |( -1 ) - ( 0 )| = 1
an |( -1 ) - ( 0 )| = 1
this |( -1 ) - ( 0 )| = 1
there |( -1 ) - ( 0 )| = 1
There |( -1 ) - ( 0 )| = 1
Perestroika |( -1 ) - ( 0 )| = 1
besuboru |( -1 ) - ( 0 )| = 1
de |( -1 ) - ( 0 )| = 1
etc. |( -1 ) - ( 0 )| = 1
of |( -1 ) - ( 0 )| = 1
for |( -1 ) - ( 0 )| = 1
on |( -1 ) - ( 0 )| = 1
new |( -1 ) - ( 0 )| = 1
other |( -1 ) - ( 0 )| = 1
last |( -1 ) - ( 0 )| = 1
many |( -1 ) - ( 0 )| = 1
higher |( -1 ) - ( 0 )| = 1
lower |( -1 ) - 

Matte's notes:
- my hunch is that GPT is struggling to differentiate between opposite (-1) and irrelevant (0). maybe this can be simplified or strengthened?

In [117]:
# surface words from axis_emb[0] that have a difference of less than 0.25
len_b = 0
for i in range(len(diff_run_list)):
  if diff_run_list[i] < 0.25:
    print(axis_emb[0][i], "|(", first_run[i], ")", "-", "(", second_run[i], ")|", "=", diff_run_list[i])
    len_b += 1

print("Number of words with a difference of less than 0.25 (very confident): ", len_b, "of", len(diff_run_list))
print("Percentage: ", round(len_b/len(diff_run_list) * 100), "%")

companion |( 0.0 ) - ( 0 )| = 0.0
toast |( 0.7 ) - ( 0.65 )| = 0.05
lounge |( -0.3 ) - ( -0.1 )| = 0.2
watch |( 0.6 ) - ( 0.7 )| = 0.1
haul |( 0.8 ) - ( 0.8 )| = 0.0
combination |( -0.2 ) - ( -0.3 )| = 0.1
extinguish |( 0.9 ) - ( 0.8 )| = 0.1
rest |( 0.6 ) - ( 0.6 )| = 0.0
hold |( 0.8 ) - ( 0.75 )| = 0.05
drainage |( -0.1 ) - ( -0.15 )| = 0.05
chill |( 0.7 ) - ( 0.65 )| = 0.05
drink |( 0.7 ) - ( 0.7 )| = 0.0
eat |( 0.8 ) - ( 0.9 )| = 0.1
journey |( 0.8 ) - ( 0.7 )| = 0.1
surf |( 0.8 ) - ( 0.8 )| = 0.0
measure |( 0.7 ) - ( 0.85 )| = 0.15
devour |( 1.0 ) - ( 0.9 )| = 0.1
catch |( 0.9 ) - ( 0.75 )| = 0.15
drive |( 0.9 ) - ( 0.95 )| = 0.05
sail |( 0.8 ) - ( 0.9 )| = 0.1
flutter |( 0.7 ) - ( 0.8 )| = 0.1
descend |( 1.0 ) - ( 0.85 )| = 0.15
commute |( 0.9 ) - ( 1.0 )| = 0.1
score |( 0.8 ) - ( 1.0 )| = 0.2
indulgence |( -0.5 ) - ( -0.5 )| = 0.0
cutting |( 0.8 ) - ( 1.0 )| = 0.2
containment |( -0.4 ) - ( -0.5 )| = 0.1
communication |( 0.2 ) - ( 0.2 )| = 0.0
transportation |( 0.6 ) - ( 0.5 )| =